In [1]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import random
import os
import datetime


In [2]:
stock ={"陽明":"2609","長榮":"2603","萬海":"2615","長榮航":"2618","遠雄港":"5607","華航":"2610","台驊投控":"2636","裕民":"2606",
        "益航":"2601" ,"東森":"2614","新興":"2605" , "四維航":"5608" , "台航":"2617" }
url_list = [f"https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID={i}&YEAR_PERIOD=9999&RPT_CAT=M_QUAR" for i in stock.values()]

In [3]:
day_now =datetime.datetime.strftime(datetime.datetime.today() ,'%Y%m%d') #現在的月份
date_Q=pd.date_range(start="20170101",end=day_now ,freq="Q" ).map(lambda x : x.strftime("%Y%m"))#季

In [4]:
headers ={"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}

In [5]:
def get_request(url , headers):
    #request
    r = requests.get(url ,headers=headers)
    r.encoding="utf-8"
    res = r.text
    soup=BeautifulSoup( res ,"lxml")
    data = soup.select_one("#divDetail")
    return data

In [6]:
#需要的最前面的時間>>> 有固定格式
def get_read_html(data ):
    #bs4 解析
    
    dfs=pd.read_html(data.prettify())[0]#prettify() : 函數將我們的data物件美化所用
    dfs.columns = dfs.iloc[0,:]
    #去除不需要的文字欄位
    index_drop=np.where(dfs=="季度")[0]
    df_new=dfs.drop(index=index_drop).reset_index().drop(columns="index")
    ROE=float(df_new.iloc[1,16])#ROE
    ROA=float(df_new.iloc[1,18])#ROA
    

   
     
    return  ROE , ROA

In [7]:
df_plus=[]
for url_OE_OR  in url_list :
    
    print(url_OE_OR)
    data = get_request(url_OE_OR ,headers )
    time.sleep(15)
    while  data is None :
        data = get_request(url_OE_OR ,headers )
        time.sleep(15)
    ROE ,ROA =get_read_html(data)
    df=pd.DataFrame({"ROE":ROE ,"ROA":ROA} ,index=date_Q).round(2) 
    df_plus.append(df)

https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2609&YEAR_PERIOD=9999&RPT_CAT=M_QUAR
https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2603&YEAR_PERIOD=9999&RPT_CAT=M_QUAR
https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2615&YEAR_PERIOD=9999&RPT_CAT=M_QUAR
https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2618&YEAR_PERIOD=9999&RPT_CAT=M_QUAR
https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=5607&YEAR_PERIOD=9999&RPT_CAT=M_QUAR
https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2610&YEAR_PERIOD=9999&RPT_CAT=M_QUAR
https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2636&YEAR_PERIOD=9999&RPT_CAT=M_QUAR
https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2606&YEAR_PERIOD=9999&RPT_CAT=M_QUAR
https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2601&YEAR_PERIOD=9999&RPT_CAT=M_QUAR
https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2614&YEAR_PERIOD=9999&RPT_CAT=M_QUAR
https://go

In [8]:
#將新的資訊加入原有的資料中
for new ,  key ,name in zip(df_plus  , stock.keys() ,stock.values()) :
    #1.新比對月份是否有重複的
    old=pd.read_csv(f"./Data_price/{key}_{name}.csv",index_col=["Unnamed: 0"])#讀入舊的dataFRAME
    new_index=int(new.index.values)
    old_index=old.tail(1).index.values
    if new_index == old_index: 
        """檢查最後ㄧ列的月份是否相同，如相同drop掉，用新的資料"""
        old.drop(index =old_index ,inplace=True )
        #2.將新的月份資料結合
        new_= pd.concat([old ,new])
        
    else:
         #2.將新的月份資料結合
        new_= pd.concat([old ,new])
    
    new_.to_csv(f"./Data_price/{key}_{name}.csv")

TypeError: only size-1 arrays can be converted to Python scalars